In [7]:
import os

# Move working directory to project root
os.chdir("..")

PROJECT_ROOT = os.getcwd()
print("Project root:", PROJECT_ROOT)

Project root: C:\Users\dtruj\Documentos\proyectos


**Author:** Dorys Trujillo  
**Project:** Legal Uncertainty Index (IIJ)  
**Data Source:** Ministry of Commerce, Industry and Tourism (MinCIT)  
**Period:** 2018–2025  

## Automated Web Scraping of MinCIT Regulatory Drafts (2018–2025)

This notebook implements an automated pipeline to collect draft decree documents published by the Colombian Ministry of Commerce, Industry and Tourism (MinCIT) between 2018 and 2025. The extracted corpus serves as the primary data source for constructing the Legal Uncertainty Index (IIJ) and represents the data ingestion layer of the project.

The workflow includes automated navigation, link extraction, file downloads, and structured local storage.

### Environment Setup and Dependencies

This step installs the required dependencies for the scraping pipeline, including Playwright (browser automation), Requests (HTTP downloads), nest_asyncio (Jupyter event loop support), and the Chromium browser engine.

⚠️ These commands only need to be executed once per environment.


In [8]:
# One-time environment setup (run only once per environment)

# Install required Python packages
# pip install playwright requests nest_asyncio

# Install Playwright browser dependencies
# playwright install chromium

### Library Imports and Core Dependencies

This section loads all core libraries required for the scraping pipeline, covering file system operations, asynchronous execution, HTTP communication, URL processing, Playwright-based browser automation, and tabular data management with pandas.

In [9]:
# System and file management
import os
import shutil

# Time and asynchronous execution
import time
import asyncio

# Text processing and pattern matching
import re

# HTTP requests and file downloads
import requests

# URL utilities for decoding and link reconstruction
from urllib.parse import unquote, urljoin

# Browser automation for dynamic web scraping
from playwright.async_api import async_playwright

# Data handling and tabular storage
import pandas as pd

### Windows Compatibility Fix

This section applies a Windows-specific event loop policy required to prevent Playwright and AsyncIO conflicts in Jupyter environments. Deprecation warnings are also suppressed to keep execution output clean. The configuration only affects Windows systems.

In [10]:
# Windows compatibility fix for Playwright

import warnings
import sys
# Suppress deprecation warnings to reduce console noise
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Apply Windows-specific event loop policy required by Playwright
if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

### Async Execution Helper (Jupyter + Playwright)

This helper enables safe execution of asynchronous Playwright tasks inside Jupyter notebooks by running coroutines in an isolated background thread with a dedicated event loop. On Windows, a Proactor event loop policy is applied to ensure proper subprocess handling.

This approach improves stability, compatibility, and error isolation when running browser automation workflows.

In [11]:
# Helper function to safely run async Playwright coroutines inside Jupyter

def run_coro_in_thread(coro):
    import threading
    import queue

    # Suppress deprecation warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    # Thread-safe queue to retrieve execution results
    q = queue.Queue()

    def _runner():
        try:
            # Apply Windows-specific Proactor event loop policy for subprocess support
            if sys.platform.startswith("win"):
                asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

            # Create and assign a new isolated event loop
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)

            try:
                result = loop.run_until_complete(coro)
                q.put(("ok", result))
            finally:
                loop.close()

        except BaseException as e:
            q.put(("err", e))

    # Run coroutine in background thread
    t = threading.Thread(target=_runner, daemon=True)
    t.start()

    # Retrieve execution result
    kind, val = q.get()

    if kind == "err":
        raise val

    return val

### Scraping Configuration and Execution Overview

This section defines the main scraping parameters and execution workflow for collecting MinCIT draft decree documents (2018–2025).

### Key Features
- Year-based navigation with multiple URL fallbacks per period.
- Automated browser-based downloads using Playwright.
- Polite crawling strategy with navigation and download delays.
- Robust error handling with retry and backoff logic.
- Automatic generation of a CSV manifest to track downloaded files and metadata.

### Output
- Downloaded documents are stored locally in year-specific folders.
- A cumulative `downloads_manifest.csv` file is created to support traceability and reproducibility.

### Execution
The pipeline is executed asynchronously through Playwright and launched inside Jupyter using a dedicated event loop wrapper to ensure platform compatibility.

In [12]:
# ================== DOWNLOAD CONFIGURATION ==================

TARGET_YEARS = range(2018, 2026)  # Includes 2017 due to site structure; target period is 2018–2025
BASE_PREFIX = "https://www.mincit.gov.co/normatividad/proyectos-de-normatividad"

# Local folder where downloaded documents will be stored
DOWNLOAD_FOLDER = os.path.join("data_raw", "mincit")

os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

# Navigation and download pacing (polite scraping)
NAV_DELAY_SEC = 1.2              # Delay between page navigations
DL_DELAY_SEC = 1.0               # Delay between individual downloads

# Download robustness parameters
MAX_CONCURRENT_DOWNLOADS = 3     # Maximum concurrent downloads (reserved for future use)
MAX_RETRIES = 3                  # Number of retry attempts per download
BACKOFF_BASE = 1.8               # Exponential backoff factor
REQUEST_TIMEOUT = 120            # Request timeout (seconds)
CHUNK_SIZE = 1024 * 256          # Streaming chunk size (256 KB)
VERIFY_SSL = True                # Keep enabled in production environments

# =============================================================

def candidate_year_urls(year: int):
    """
    Generate candidate landing page URLs for a given year.
    The MinCIT website uses different naming patterns (singular/plural and historical paths).
    """
    return [
        f"{BASE_PREFIX}/proyectos-de-decreto-{year}",
        f"{BASE_PREFIX}/proyectos-de-decretos-{year}",
        f"{BASE_PREFIX}/proyecto-de-decretos-{year}",
        f"{BASE_PREFIX}/historial-proyectos/proyectos-de-decreto-{year}",
        f"{BASE_PREFIX}/historial-proyectos/proyectos-de-decretos-{year}",
    ]

def sanitize_filename(name: str) -> str:
    """
    Remove invalid filesystem characters to ensure cross-platform compatibility.
    """
    name = re.sub(r'[\\/*?:"<>|]', "_", name)
    return name.strip()

def filename_from_cd(cd_header: str) -> str | None:
    """
    Extract filename from the Content-Disposition HTTP header, if available.
    """
    if not cd_header:
        return None

    m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^\";]+)"?', cd_header, re.IGNORECASE)
    if not m:
        return None

    return sanitize_filename(unquote(m.group(1)))

def is_pdf_response(resp: requests.Response) -> bool:
    """
    Check whether the HTTP response corresponds to a PDF file.
    """
    ct = (resp.headers.get("Content-Type") or "").lower()
    return "application/pdf" in ct or ct.endswith("/pdf")

async def polite_sleep(seconds: float):
    """
    Introduce controlled delays to reduce server load and avoid aggressive scraping.
    """
    await asyncio.sleep(seconds)

async def download_pdf(url: str, out_path: str, session: requests.Session) -> dict | None:
    """
    Download a PDF file using HTTP streaming and return metadata.
    Returns None if the download fails.
    """
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            with session.get(
                url,
                timeout=REQUEST_TIMEOUT,
                stream=True,
                verify=VERIFY_SSL
            ) as r:

                r.raise_for_status()

                # Validate PDF content type (fallback: allow .pdf extension)
                if not is_pdf_response(r) and not url.lower().endswith(".pdf"):
                    print(f"    ! Non-PDF content detected (Content-Type={r.headers.get('Content-Type')}), skipping.")
                    return None

                cd_name = filename_from_cd(r.headers.get("Content-Disposition", ""))
                final_path = os.path.join(os.path.dirname(out_path), cd_name) if cd_name else out_path

                total = 0
                with open(final_path, "wb") as f:
                    for chunk in r.iter_content(chunk_size=CHUNK_SIZE):
                        if chunk:
                            f.write(chunk)
                            total += len(chunk)

                size_mb = total / (1024 * 1024)
                print(f"    ✓ Saved {os.path.basename(final_path)} ({size_mb:.2f} MB)")

                return {
                    "saved_path": os.path.abspath(final_path),
                    "bytes": total,
                    "filename": os.path.basename(final_path),
                    "source_url": url,
                }

        except Exception as e:
            if attempt < MAX_RETRIES:
                wait = (BACKOFF_BASE ** (attempt - 1))
                print(f"    ! Attempt {attempt}/{MAX_RETRIES} failed: {e} -> retrying in {wait:.1f}s")
                await asyncio.sleep(wait)
            else:
                print(f"    ✗ Download failed after {MAX_RETRIES} attempts: {e}")
                return None

async def scrape_year(page, year: int) -> int:
    """
    Scrape all available draft decree documents for a given year.
    Returns the number of successfully downloaded files.
    """
    downloaded = 0
    manifest_rows = []  # Accumulate metadata rows for the current year

    for url in candidate_year_urls(year):
        try:
            await page.goto(url, timeout=90000, wait_until="load")
            await polite_sleep(NAV_DELAY_SEC)

            # Scroll to trigger lazy-loaded content
            await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
            await polite_sleep(0.6)

            # Primary selector for "Ver documento" links
            anchors = await page.get_by_role(
                "link",
                name=re.compile(r"ver\s*documento", re.I)
            ).all()

            if not anchors:
                anchors = await page.locator('a:has-text("Ver documento")').all()

            if not anchors:
                # Fallback selector for any link/button mentioning "documento"
                anchors = await page.locator(
                    'a, button, [role="link"]',
                    has_text=re.compile(r"documento", re.I)
                ).all()

            if not anchors:
                print(f"  -> {year}: no 'Ver documento' links found at {url}")
                continue

            print(f"  -> {year}: {len(anchors)} documents found at {url}")

            year_folder = os.path.join(DOWNLOAD_FOLDER, f"{year}")
            os.makedirs(year_folder, exist_ok=True)

            # Sequential downloads (human-like behavior)
            for i, a in enumerate(anchors, start=1):
                await polite_sleep(DL_DELAY_SEC)

                try:
                    # Wait for browser download event triggered by click
                    async with page.expect_download(timeout=60000) as dl_info:
                        await a.click(button="left")

                    download = await dl_info.value

                    suggested = download.suggested_filename or f"decreto_{i}.pdf"
                    fname = sanitize_filename(suggested)
                    final_path = os.path.join(year_folder, fname)

                    await download.save_as(final_path)

                    size_bytes = os.path.getsize(final_path)
                    downloaded += 1

                    print(f"    ✓ Saved {fname} ({size_bytes / 1024 / 1024:.2f} MB)")

                    manifest_rows.append({
                        "saved_path": os.path.abspath(final_path),
                        "bytes": size_bytes,
                        "filename": fname,
                        "source_url": download.url or "",
                        "year": int(year),
                        "landing_page": url,
                    })

                except Exception as e:
                    print(f"    ! Download failed for link #{i}: {e}")

            # Append yearly batch to manifest CSV
            if manifest_rows:
                df = pd.DataFrame(manifest_rows)

                csv_path = os.path.join(DOWNLOAD_FOLDER, "downloads_manifest.csv")
                header = not os.path.exists(csv_path)

                df.to_csv(
                    csv_path,
                    mode="a",
                    header=header,
                    index=False,
                    encoding="utf-8"
                )

                print(f"  -> Manifest updated: {csv_path} (+{len(df)} rows)")

            # Stop searching alternative URLs if this one succeeded
            return downloaded

        except Exception as e:
            print(f"  -> {year}: error while processing {url}: {e}")

    print(f"  -> {year}: no valid landing page found.")
    return downloaded

async def main():
    """
    Main orchestration function that launches the browser,
    iterates over target years, and aggregates download statistics.
    """
    total = 0

    async with async_playwright() as p:

        browser = await p.chromium.launch(headless=True)

        context = await browser.new_context(
            user_agent=(
                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                "(KHTML, like Gecko) Chrome/126.0 Safari/537.36"
            ),
            ignore_https_errors=True
        )

        page = await context.new_page()

        for year in TARGET_YEARS:
            print(f"\n=== YEAR {year} ===")
            count = await scrape_year(page, year)
            total += count

        await browser.close()

    print("\n--- PROCESS COMPLETED ---")
    print(f"Total documents downloaded: {total}")
    print(f"Output folder: {DOWNLOAD_FOLDER}")

# Entry point for execution inside Jupyter/Colab environments
# await main()
run_coro_in_thread(main())


=== YEAR 2018 ===
  -> 2018: no 'Ver documento' links found at https://www.mincit.gov.co/normatividad/proyectos-de-normatividad/proyectos-de-decreto-2018
  -> 2018: no 'Ver documento' links found at https://www.mincit.gov.co/normatividad/proyectos-de-normatividad/proyectos-de-decretos-2018
  -> 2018: 32 documents found at https://www.mincit.gov.co/normatividad/proyectos-de-normatividad/proyecto-de-decretos-2018
    ✓ Saved DECRETO-NUMERO-DE-2018.pdf (0.42 MB)
    ✓ Saved DECRETO-2.pdf (0.74 MB)
    ✓ Saved DECRETO-3.pdf (0.46 MB)
    ✓ Saved image2019-02-25-104030.pdf (5.95 MB)
    ✓ Saved DECRETO-5.pdf (0.30 MB)
    ✓ Saved DECRETO-6.pdf (0.42 MB)
    ✓ Saved DECRETO-7.pdf (0.30 MB)
    ✓ Saved DECRETO-8.pdf (0.32 MB)
    ✓ Saved DECRETO-9.pdf (0.39 MB)
    ✓ Saved DECRETO-10.pdf (0.20 MB)
    ✓ Saved DECRETO-11.pdf (0.42 MB)
    ✓ Saved DECRETO-12.pdf (0.49 MB)
    ✓ Saved DECRETO-13.pdf (0.44 MB)
    ✓ Saved DECRETO-14.pdf (0.23 MB)
    ✓ Saved DECRETO-15.pdf (0.23 MB)
    ✓ Saved 

#### Download Manifest Analysis and Annual File Size Aggregation

The script loads the downloads_manifest.csv file with explicit typing and selected columns, filters out invalid records, and computes the total number of downloaded documents. It aggregates downloads by year to obtain annual counts and identifies, for each year, the largest file based on the bytes field, with an optional extraction of the top-N largest files per year.

In [15]:
import pandas as pd

# Define explicit data types to ensure memory efficiency and schema consistency
dtypes = {
    "saved_path": "string",
    "bytes": "Int64",
    "filename": "string",
    "source_url": "string",
    "year": "Int64",
    "landing_page": "string"
}

# Load only relevant columns from the downloads manifest
df = pd.read_csv(
    "legal-uncertainty-index/notebooks/downloads_manifest.csv",
    dtype=dtypes,
    usecols=["filename", "bytes", "year"]
)

# Remove records with missing year or file size information
df = df.dropna(subset=["year", "bytes"])

# Compute total number of valid downloaded documents
total_documents = len(df)
print(f"Total downloaded documents: {total_documents}")

Total downloaded documents: 334


In [16]:
# Compute the number of downloaded documents per year
documents_per_year = (
    df.groupby("year", observed=True)
      .size()
      .reset_index(name="num_documents")
      .sort_values("year")
)

print(documents_per_year)

   year  num_documents
0  2018             32
1  2019             63
2  2020             49
3  2021             51
4  2022             29
5  2023             32
6  2024             35
7  2025             43


In [17]:
# Identify the largest downloaded file (by size in bytes) for each year
idx = df.groupby("year")["bytes"].idxmax()

largest_file_per_year = (
    df.loc[idx, ["year", "filename", "bytes"]]
      .sort_values("year")
      .reset_index(drop=True)
)

print(largest_file_per_year)

   year                                           filename      bytes
0  2018                         image2019-02-25-104030.pdf    6244212
1  2019                            Proyecto-de-decreto.pdf    9048357
2  2020                      13-11-20-PD-ESTRUCTURA-SS.pdf   15255210
3  2021                       08-11-21-PD-VII-Enmienda.pdf  266375040
4  2022  18-02-2022-Proyecto-Decreto-Implementacion-tlc...    1329899
5  2023  29-08-2023-PD-Incorporacion-NIIF-17-contrato-d...    1033605
6  2024  10-05-2024-PD-Certificado-de-Reembolso-Tributa...    1131859
7  2025  21-11-2025-PD-modifica-en-lo-relativo-a-las-bu...     715439


In [18]:
# Retrieve the top N largest files per year based on file size
top_n = 3

top_n_files_per_year = (
    df.sort_values(["year", "bytes"], ascending=[True, False])
      .groupby("year")
      .head(top_n)
      .reset_index(drop=True)
)

print(top_n_files_per_year)

        bytes                                           filename  year
0     6244212                         image2019-02-25-104030.pdf  2018
1      777057                                      DECRETO-2.pdf  2018
2      637484                                     DECRETO-32.pdf  2018
3     9048357                            Proyecto-de-decreto.pdf  2019
4     1039000  Proyecto-decreto-desgravacion-arancelaria-TLC-...  2019
5      830956                        PytoD-ActualizacionD272.pdf  2019
6    15255210                      13-11-20-PD-ESTRUCTURA-SS.pdf  2020
7     5442937          26-06-20-Decreto-Astilleros-27jun2020.pdf  2020
8     3817223  31-08-20-PD-Trazabilidad-de-Alcohol-Potable-y-...  2020
9   266375040                       08-11-21-PD-VII-Enmienda.pdf  2021
10    9500587            17-11-21-PD-Derechos-Compensatorios.pdf  2021
11    5814224                                28-09-21-PD-RNT.pdf  2021
12    1329899  18-02-2022-Proyecto-Decreto-Implementacion-tlc...  2022
13    

In [19]:
import os
from pathlib import Path

PROJECT_ROOT = Path(PROJECT_ROOT)
# Root directory containing yearly subfolders
DOWNLOAD_FOLDER = PROJECT_ROOT / "data_raw" / "mincit"

# Validate directory existence
if not DOWNLOAD_FOLDER.exists():
    raise FileNotFoundError(f"Download folder not found: {DOWNLOAD_FOLDER}")

pdf_counts = {}

# Iterate over year-level subdirectories
for subfolder in sorted(DOWNLOAD_FOLDER.iterdir()):
    if subfolder.is_dir():
        num_pdfs = len(list(subfolder.glob("*.pdf")))
        pdf_counts[subfolder.name] = num_pdfs

# Display results
for year, count in pdf_counts.items():
    print(f"{year}: {count} pdf")

2018: 32 pdf
2019: 59 pdf
2020: 49 pdf
2021: 51 pdf
2022: 29 pdf
2023: 32 pdf
2024: 35 pdf
2025: 43 pdf
